In [65]:
import cyecca
import numpy as np
import casadi as ca
from cyecca.lie.group_so3 import SO3Quat, so3

def derive_cov_prop():
    chi = so3.elem(ca.SX.sym('chi', 3))
    wb = so3.elem(ca.SX.sym('wb', 3))
    A = -wb.ad()
    #B = chi.right_jacobian_inv()

    P0 = ca.SX.sym('P0', 3, 3)
    Q = ca.SX.sym('Q', 3, 3)
    dt = ca.SX.sym('dt')

    # prediction
    P1 = P0 + A @ P0 + P0 @ A.T + Q

    f_cov_prop = ca.Function('cov_prop', [P0, dt, wb.param, Q], [P1], ['P0', 'dt', 'wb', 'Q'], ['P1'])

    return {
        'f_cov_prop': f_cov_prop,
    }


res = derive_cov_prop()

In [71]:
P = np.eye(3)
dt = 0.01
w = np.array([1, 2, 3])
Q = np.eye(3)*0.1
for i in range(1000):
    P = res["f_cov_prop"](P, dt, w, Q)
P

DM(
[[101, 0, 0], 
 [0, 101, 0], 
 [0, 0, 101]])

In [80]:
np.block([
    [np.eye(3), np.zeros((3, 3))],
    [np.zeros((3, 3)), np.eye(3)]
    ])

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [ ]:
np.block([])